# LightFM runbook

## Loading libraries and setting parameters

In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys

sys.path.append("../../..")

from src.common import helpers, config
from src.models.lightfm import LFM
from src.evaluation.evaluator import Evaluator

In [2]:
MODEL_NAME = "lightfm"

paths = config.Paths(
    dataset_name="jobs_published",
    target_users_name="subset_validation",
    model_name=MODEL_NAME,
)

In [3]:
no_components=200
learning_schedule="adadelta"
loss="warp"
max_sampled=61
epochs=11

N_RECOMMENDATIONS = 10

## Model training

In [4]:
model = LFM(
    no_components=no_components,
    learning_schedule=learning_schedule,
    loss=loss,
    max_sampled=max_sampled,
    epochs=epochs,
)

model.load_interactions(paths.train)

model.preprocess()

model.fit()

Epoch: 100%|████████████████████████████████████| 11/11 [02:18<00:00, 12.56s/it]


## Generating recommendations

In [5]:
target_users = model.load_target_users(paths.target_users)

In [6]:
recommendations = model.recommend(
    target_users=target_users, n_recommendations=N_RECOMMENDATIONS, show_progress=True
)

model.save_recommendations(recommendations, paths.recommendations)

30000it [00:51, 578.66it/s]


In [7]:
evaluator = Evaluator(
    recommendations_path=paths.recommendations_folder,
    test_path=paths.validation,
    k=10,
    models_to_evaluate=[MODEL_NAME],
)

evaluator.prepare()

evaluator.evaluate_models()

30000it [00:02, 12506.49it/s]


In [8]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
lightfm,0.024273,0.100587,0.035434,0.027474,0.067663,0.042885,0.083395,0.55002,0.194733,0.977673,0.67858,8.42264,0.795029,0.0,0.0


In [9]:
evaluator.evaluation_results

,precision,recall,F_1,F_05,ndcg,mAP,MRR,LAUC,HR,reco_in_test,test_coverage,Shannon,Gini,users_without_reco,users_without_k_reco
model_name,,,,,,,,,,,,,,,
lightfm,0.024273,0.100587,0.035434,0.027474,0.067663,0.042885,0.083395,0.55002,0.194733,0.977673,0.67858,8.42264,0.795029,0.0,0.0
